<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_N1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [10]:
%%writefile tp1.cpp
#include <iostream>
#include <unistd.h>
  
int main()
{
    pid_t c_pid = fork();
  
    if (c_pid == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    } else if (c_pid) {
        //  wait(nullptr);
        std::cout << "Hola, soy el padre " << getpid() << std::endl;
    } else {
        std::cout << "Hola, soy el hijo " << getpid() << std::endl;
    }
  
    return 0;
}

Overwriting tp1.cpp


In [11]:
!g++ tp1.cpp && ./a.out

Hola, soy el padre 1891
Hola, soy el hijo 1897
